In [ ]:
import numpy as np
import torch

In [ ]:
npy_noisy = np.load('../feature/noisy.npy')
npy_clean = np.load('../feature/clean.npy')

In [ ]:
display(npy_noisy.shape)
display(type(npy_noisy))

In [ ]:
torch_noisy = torch.from_numpy(npy_noisy)
torch_clean = torch.from_numpy(npy_clean)

In [ ]:
display(torch_noisy.shape)
display(type(torch_noisy))

In [ ]:
data = torch.stack((torch_noisy,torch_clean),0)

In [ ]:
display(data.shape)

In [ ]:
m = torch.nn.Linear()

# DatasetDCUNET

In [1]:
from dataset.DCUNET import DatasetDCUNET
from utils.hparams import HParam
from model.DCUNET import ModelDCUNET
import torch

/home/kiosk/anaconda3/envs/dnn/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
f = open('../config/default.yaml')
f.read()

"gpu: 'cuda:0'\n---\nmodel :\n  FC :\n    len : 3\n  DCUNET : \n    num_frame : 80\n---\ntrain:\n  epoch: 50\n  # 12\n  batch_size: 12\n  # 16\n  num_workers: 16\n  optimizer: 'adam'\n  adam: 0.0001\n  adabound:\n    initial: 0.001\n    final: 0.05\n  frame_num: 1024\n\n  summary_interval: 10\n  validation_interval: 100\n  checkpoint_interval: 1000\n---\nloss:\n  type: 'wSDR'\n---\nscheduler: # Plateau, oneCycle, None\n  type: 'Plateau'\n  oneCycle:\n    max_lr: 0.0001\n  Plateau:\n    mode: 'min'\n    patience: 2\n    factor: 0.5\n    min_lr: 0.000005\n---\naudio:\n  frame : 1024\n  shift : 256\n  samplerate : 16000\n---\ndata:\n  root: '/home/data/3-channel-dnn/'\n---\n  log:\n    root: '/home/nas/user/kbh/3-channel-dnn/'\n"

In [3]:
hp = HParam('../config/default.yaml')

In [4]:
data = DatasetDCUNET(hp.data.root+'/STFT_R',hp.data.root+'/WAV',['tr05_bus_simu','tr05_caf_simu','tr05_ped_simu','tr05_str_simu'],'*.npy',num_frame=hp.model.DCUNET.num_frame)

In [5]:
x= data.__getitem__(1568)
print(x['input'].shape)
print(x['clean'].shape)
print(x['noisy'].shape)

75
20032
20032
-5
21312
clean
tr05_bus_simu/20A_20AC010G_BUS
1568
noisy
tr05_bus_simu/20A_20AC010G_BUS
1568
torch.Size([3, 513, 80, 2])
torch.Size([21312])
torch.Size([21312])


In [6]:
input = x['input']
# Batch : 1
input =torch.unsqueeze(input,0)
print(input.shape)

torch.Size([1, 3, 513, 80, 2])


In [7]:
model = ModelDCUNET(input_channels=3)

In [8]:
print(input.shape)
output = model(input)

torch.Size([1, 3, 513, 80, 2])


# DatasetFC : How?

+ Magnitude only ? 
+ Respectively ? 

In [7]:
from dataset import DatasetFC
import torch.nn as nn

In [2]:
d = DatasetFC('/home/data/kbh/3-channel-dnn/STFT_R/','dt05_bus_simu','*.npy')
a = d[0]

In [9]:
display(a['input'].shape)

torch.Size([3, 513, 7, 2])